In [ ]:
!pip install shap lime transformers datasets
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

model_path = "final_amharic_ner_model"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()


In [ ]:
def predict_ner_labels(texts):
    encoded = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, is_split_into_words=False)
    with torch.no_grad():
        output = model(**encoded).logits
    predictions = torch.argmax(output, dim=-1).squeeze().tolist()
    tokens = tokenizer.convert_ids_to_tokens(encoded["input_ids"].squeeze().tolist())
    return list(zip(tokens, predictions))


In [ ]:
import shap

def shap_ner(texts):
    def f(X):
        inputs = tokenizer(X, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs).logits
        return outputs.detach().numpy()

    explainer = shap.Explainer(f, tokenizer)
    shap_values = explainer(texts)
    shap.plots.text(shap_values[0])  # visual explanation of the first sentence

shap_ner(["አዲስ ስልኮች በ15000 ብር"])


In [ ]:
from lime.lime_text import LimeTextExplainer

class_names = list(id2label.values())

def lime_ner_explain(text):
    explainer = LimeTextExplainer(class_names=class_names)
    
    def predict_fn(text_list):
        results = []
        for t in text_list:
            tokenized = tokenizer(t, return_tensors="pt", truncation=True, padding=True)
            output = model(**tokenized).logits
            probs = torch.nn.functional.softmax(output, dim=-1)
            results.append(probs.squeeze().detach().numpy()[0])  # probability of first token
        return results

    exp = explainer.explain_instance(text, predict_fn, num_features=10)
    exp.show_in_notebook()

lime_ner_explain("የህፃናት ጫማ በ500 ብር")
